In [1]:
import random

companies = ["Accenture", "Wippro", "NCS", "Avepoint","KPMG","Cognizant","Deloitte"]
roles = ["software engineer", "ux designer", "business analyst", "full stack developer", "delivery manager","business transformer","data scientist"]
seniorities = ["associate", "engineer", "senior engineer"]

def generate_cost(seniority):
    if seniority == "Associate":
        return random.randint(50000, 65000)
    elif seniority == "Engineer":
        return random.randint(70000, 85000)
    else:
        return random.randint(90000, 105000)

with open("company_data.csv", "w") as f:
    f.write("Company,Role,Seniority,Cost\n")
    for company in companies:
        for role in roles:
            seniority = random.choice(seniorities)
            cost = generate_cost(seniority)
            f.write(f"{company},{role},{seniority},{cost}\n")

In [2]:
import sqlite3

def load_data_to_database(csv_file, database_file):
    conn = sqlite3.connect(database_file)
    c = conn.cursor()

    # Create a table if it doesn't exist
    c.execute('''CREATE TABLE IF NOT EXISTS company_data
                 (company TEXT, role TEXT, seniority TEXT, cost INTEGER)''')

    with open(csv_file, 'r') as f:
        for line in f:
            if line.startswith('Company'):
                continue  # Skip header row
            company, role, seniority, cost = line.strip().split(',')
            c.execute("INSERT INTO company_data VALUES (?, ?, ?, ?)",
                     (company, role, seniority, int(cost.replace('$', ''))))

    conn.commit()
    conn.close()

csv_file = "company_data.csv"
database_file = "company_data.db"
load_data_to_database(csv_file, database_file)

In [3]:
import sqlite3

def list_database_entries(database_file):
    conn = sqlite3.connect(database_file)
    c = conn.cursor()

    c.execute("SELECT * FROM company_data")
    rows = c.fetchall()

    for row in rows:
        print(row)

    conn.close()

database_file = "company_data.db"
response = list_database_entries(database_file)



('Accenture', 'software engineer', 'engineer', 92393)
('Accenture', 'ux designer', 'engineer', 100607)
('Accenture', 'business analyst', 'associate', 92084)
('Accenture', 'full stack developer', 'associate', 98029)
('Accenture', 'delivery manager', 'associate', 92266)
('Accenture', 'business transformer', 'engineer', 94501)
('Accenture', 'data scientist', 'associate', 98714)
('Wippro', 'software engineer', 'engineer', 100753)
('Wippro', 'ux designer', 'senior engineer', 92147)
('Wippro', 'business analyst', 'engineer', 93994)
('Wippro', 'full stack developer', 'senior engineer', 97637)
('Wippro', 'delivery manager', 'associate', 91819)
('Wippro', 'business transformer', 'engineer', 90377)
('Wippro', 'data scientist', 'associate', 100471)
('NCS', 'software engineer', 'associate', 104214)
('NCS', 'ux designer', 'engineer', 95275)
('NCS', 'business analyst', 'associate', 103618)
('NCS', 'full stack developer', 'senior engineer', 100949)
('NCS', 'delivery manager', 'senior engineer', 91415

In [ ]:
import toml
import os

with open(".streamlit/secrets.toml", "r") as f:
    secrets = toml.load(f)

#print(secrets['OPENAI_KEY'])
os.environ['OPENAI_API_KEY'] = secrets['OPENAI_KEY']

In [ ]:
from openai import OpenAI
from getpass import getpass
import toml


client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

def get_completion(prompt, model="gpt-4o-mini"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content

response = get_completion("Hello")
response

In [ ]:
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew
from crewai_tools import tool
import sqlite3

@tool("SQLreader")
def readsqltable (query: str) -> str:
    """
    Reads a table from the SQLite database.

    Args:
        query (str): The SQL query to execute.

    Returns:
        None
    """
    # Connect to the SQLite database
    conn = sqlite3.connect('company_data.db')

    # Create a cursor object
    cur = conn.cursor()

    # Execute a query to select data from a table
    cur.execute(query)

    # Fetch all rows from the query
    rows = cur.fetchall()

    # Print the rows
    #for row in rows:
    #    print(row)

    # Close the connection
    conn.close()
    
    return rows


# Change the working directory to the current file's directory#
#os.chdir(os.path.dirname(os.path.abspath(__file__)))
#print(f"The current working directory is: {os.getcwd()}")

load_dotenv('.env')


agent_SQL = Agent(
    role="Database expert",
    goal="Generate an SQL statement for {question} based on the questions provided.",

    backstory="""
    You are a database expert, who is responsible for generating a database query for a specific database.
    """,
    allow_delegation=False, 
	verbose=True, 
)

agent_writer = Agent(
    role=" Writer",
    goal="Analyse the data from the SQL results. ",
    backstory="""Analyse the data generated from the SQL statement. """,
    allow_delegation=False, 
    verbose=True, 
)

task_sql = Task(
    description="""\
    1. The database has a table company_data with columns "company", "role", "seniority", and "cost"
    2. Ensure the accuracy of the SQL statment generated by checking the SQL statement against the table.
    3. Ensure the data generated will be informative.
    """,

    expected_output="""\
    Am accurate SQL statement that is correct.""",
    agent=agent_SQL, 
    tools=[readsqltable],
    context=[]
)

task_write = Task(
    description="""\
    1. Create the table of the input data generated by the database expert
    2. The data given is to assist in hiring of resources for projects. 
    3. Analyse the data through a comparing the data and providing insight using agent_SQL onlyt.
    """,

    expected_output="""
    An analysis of the data generated.""",
    agent=agent_writer,
)


crew = Crew(
    agents=[agent_SQL, agent_writer],
    tasks=[task_sql, task_write],
    verbose=True
)

result = crew.kickoff(inputs={"question": "I want to hire 2 Software Engineer for a project. Find the cheapest and give me the details."})


print(f"Raw Output: {result.raw}")
print("-----------------------------------------\n\n")
print(f"Token Usage: {result.token_usage}")
print("-----------------------------------------\n\n")
print(f"Tasks Output of Task 1: {result.tasks_output[0]}")
print("-----------------------------------------\n\n")
print(f"Tasks Output of Task 2: {result.tasks_output[1]}")

In [ ]:
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew
from crewai_tools import tool
import sqlite3

@tool("SQLreader")
def readsqltable (query: str) -> str:
    """
    Reads a table from the SQLite database.

    Args:
        query (str): The SQL query to execute.

    Returns:
        None
    """
    # Connect to the SQLite database
    conn = sqlite3.connect('company_data.db')

    # Create a cursor object
    cur = conn.cursor()

    # Execute a query to select data from a table
    cur.execute(query)

    # Fetch all rows from the query
    rows = cur.fetchall()

    # Print the rows
    #for row in rows:
    #    print(row)

    # Close the connection
    conn.close()
    
    return rows


# Change the working directory to the current file's directory#
#os.chdir(os.path.dirname(os.path.abspath(__file__)))
#print(f"The current working directory is: {os.getcwd()}")

load_dotenv('.env')


agent_SQL = Agent(
    role="Database expert",
    goal="Generate an SQL statement for {question} based on the questions provided.",

    backstory="""
    You are a database expert, who is responsible for generating a database query for a specific database.
    """,
    allow_delegation=False, 
	verbose=True, 
)

agent_writer = Agent(
    role=" Writer",
    goal="Analyse the data from the SQL results. ",
    backstory="""Analyse the data generated from the SQL statement. """,
    allow_delegation=False, 
    verbose=True, 
)

task_sql = Task(
    description="""\
    1. The database has a table company_data with columns "company", "role", "seniority", and "cost"
    2. Ensure the accuracy of the SQL statment generated by checking the SQL statement against the table.
    3. Ensure the data generated will be informative.
    """,

    expected_output="""\
    Am accurate SQL statement that is correct.""",
    agent=agent_SQL, 
    tools=[readsqltable],
    context=[]
)

task_write = Task(
    description="""\
    1. Create the table of the input data generated by the database expert
    2. The data given is to assist in hiring of resources for projects. 
    3. Analyse the data through a comparing the data and providing insight using agent_SQL onlyt.
    """,

    expected_output="""
    An analysis of the data generated.""",
    agent=agent_writer,
)


crew = Crew(
    agents=[agent_SQL, agent_writer],
    tasks=[task_sql, task_write],
    verbose=True
)

result = crew.kickoff(inputs={"question": "I want to hire 2 Software Engineer for a project. Find the cheapest and give me the details."})


print(f"Raw Output: {result.raw}")
print("-----------------------------------------\n\n")
print(f"Token Usage: {result.token_usage}")
print("-----------------------------------------\n\n")
print(f"Tasks Output of Task 1: {result.tasks_output[0]}")
print("-----------------------------------------\n\n")
print(f"Tasks Output of Task 2: {result.tasks_output[1]}")

In [ ]:
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew
from crewai_tools import tool
import sqlite3

@tool("SQLreader")
def readsqltable (query: str) -> str:
    """
    Reads a table from the SQLite database.

    Args:
        query (str): The SQL query to execute.

    Returns:
        None
    """
    # Connect to the SQLite database
    conn = sqlite3.connect('company_data.db')

    # Create a cursor object
    cur = conn.cursor()

    # Execute a query to select data from a table
    cur.execute(query)

    # Fetch all rows from the query
    rows = cur.fetchall()

    # Print the rows
    #for row in rows:
    #    print(row)

    # Close the connection
    conn.close()
    
    return rows


# Change the working directory to the current file's directory#
#os.chdir(os.path.dirname(os.path.abspath(__file__)))
#print(f"The current working directory is: {os.getcwd()}")

load_dotenv('.env')


agent_SQL = Agent(
    role="Database expert",
    goal="Generate an SQL statement for {question} based on the questions provided.",

    backstory="""
    You are a database expert, who is responsible for generating a database query for a specific database.
    """,
    allow_delegation=False, 
	verbose=True, 
)

agent_writer = Agent(
    role=" Writer",
    goal="Analyse the data from the SQL results. ",
    backstory="""Analyse the data generated from the SQL statement. """,
    allow_delegation=False, 
    verbose=True, 
)

task_sql = Task(
    description="""\
    1. The database has a table company_data with columns "company", "role", "seniority", and "cost"
    2. Ensure the accuracy of the SQL statment generated by checking the SQL statement against the table.
    3. Ensure the data generated will be informative.
    """,

    expected_output="""\
    Am accurate SQL statement that is correct.""",
    agent=agent_SQL, 
    tools=[readsqltable],
    context=[]
)

task_write = Task(
    description="""\
    1. Create the table of the input data generated by the database expert
    2. The data given is to assist in hiring of resources for projects. 
    3. Analyse the data through a comparing the data and providing insight using agent_SQL onlyt.
    """,

    expected_output="""
    An analysis of the data generated.""",
    agent=agent_writer,
)


crew = Crew(
    agents=[agent_SQL, agent_writer],
    tasks=[task_sql, task_write],
    verbose=True
)

result = crew.kickoff(inputs={"question": "I want to hire 2 Software Engineer for a project. Find the cheapest and give me the details."})


print(f"Raw Output: {result.raw}")
print("-----------------------------------------\n\n")
print(f"Token Usage: {result.token_usage}")
print("-----------------------------------------\n\n")
print(f"Tasks Output of Task 1: {result.tasks_output[0]}")
print("-----------------------------------------\n\n")
print(f"Tasks Output of Task 2: {result.tasks_output[1]}")

In [ ]:
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew
from crewai_tools import tool
import sqlite3

@tool("SQLreader")
def readsqltable (query: str) -> str:
    """
    Reads a table from the SQLite database.

    Args:
        query (str): The SQL query to execute.

    Returns:
        None
    """
    # Connect to the SQLite database
    conn = sqlite3.connect('company_data.db')

    # Create a cursor object
    cur = conn.cursor()

    # Execute a query to select data from a table
    cur.execute(query)

    # Fetch all rows from the query
    rows = cur.fetchall()

    # Print the rows
    #for row in rows:
    #    print(row)

    # Close the connection
    conn.close()
    
    return rows


# Change the working directory to the current file's directory#
#os.chdir(os.path.dirname(os.path.abspath(__file__)))
#print(f"The current working directory is: {os.getcwd()}")

load_dotenv('.env')


agent_SQL = Agent(
    role="Database expert",
    goal="Generate an SQL statement for {question} based on the questions provided.",

    backstory="""
    You are a database expert, who is responsible for generating a database query for a specific database.
    """,
    allow_delegation=False, 
	verbose=True, 
)

agent_writer = Agent(
    role=" Writer",
    goal="Analyse the data from the SQL results. ",
    backstory="""Analyse the data generated from the SQL statement. """,
    allow_delegation=False, 
    verbose=True, 
)

task_sql = Task(
    description="""\
    1. The database has a table company_data with columns "company", "role", "seniority", and "cost"
    2. Ensure the accuracy of the SQL statment generated by checking the SQL statement against the table.
    3. Ensure the data generated will be informative.
    """,

    expected_output="""\
    Am accurate SQL statement that is correct.""",
    agent=agent_SQL, 
    tools=[readsqltable],
    context=[]
)

task_write = Task(
    description="""\
    1. Create the table of the input data generated by the database expert
    2. The data given is to assist in hiring of resources for projects. 
    3. Analyse the data through a comparing the data and providing insight using agent_SQL onlyt.
    """,

    expected_output="""
    An analysis of the data generated.""",
    agent=agent_writer,
)


crew = Crew(
    agents=[agent_SQL, agent_writer],
    tasks=[task_sql, task_write],
    verbose=True
)

result = crew.kickoff(inputs={"question": "I want to hire 2 Software Engineer for a project. Find the cheapest and give me the details."})


print(f"Raw Output: {result.raw}")
print("-----------------------------------------\n\n")
print(f"Token Usage: {result.token_usage}")
print("-----------------------------------------\n\n")
print(f"Tasks Output of Task 1: {result.tasks_output[0]}")
print("-----------------------------------------\n\n")
print(f"Tasks Output of Task 2: {result.tasks_output[1]}")

In [ ]:
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew
from crewai_tools import tool
import sqlite3

@tool("SQLreader")
def readsqltable (query: str) -> str:
    """
    Reads a table from the SQLite database.

    Args:
        query (str): The SQL query to execute.

    Returns:
        None
    """
    # Connect to the SQLite database
    conn = sqlite3.connect('company_data.db')

    # Create a cursor object
    cur = conn.cursor()

    # Execute a query to select data from a table
    cur.execute(query)

    # Fetch all rows from the query
    rows = cur.fetchall()

    # Print the rows
    #for row in rows:
    #    print(row)

    # Close the connection
    conn.close()
    
    return rows


# Change the working directory to the current file's directory#
#os.chdir(os.path.dirname(os.path.abspath(__file__)))
#print(f"The current working directory is: {os.getcwd()}")

load_dotenv('.env')


agent_SQL = Agent(
    role="Database expert",
    goal="Generate an SQL statement for {question} based on the questions provided.",

    backstory="""
    You are a database expert, who is responsible for generating a database query for a specific database.
    """,
    allow_delegation=False, 
	verbose=True, 
)

agent_writer = Agent(
    role=" Writer",
    goal="Analyse the data from the SQL results. ",
    backstory="""Analyse the data generated from the SQL statement. """,
    allow_delegation=False, 
    verbose=True, 
)

task_sql = Task(
    description="""\
    1. The database has a table company_data with columns "company", "role", "seniority", and "cost"
    2. Ensure the accuracy of the SQL statment generated by checking the SQL statement against the table.
    3. Ensure the data generated will be informative.
    """,

    expected_output="""\
    Am accurate SQL statement that is correct.""",
    agent=agent_SQL, 
    tools=[readsqltable],
    context=[]
)

task_write = Task(
    description="""\
    1. Create the table of the input data generated by the database expert
    2. The data given is to assist in hiring of resources for projects. 
    3. Analyse the data through a comparing the data and providing insight using agent_SQL onlyt.
    """,

    expected_output="""
    An analysis of the data generated.""",
    agent=agent_writer,
)


crew = Crew(
    agents=[agent_SQL, agent_writer],
    tasks=[task_sql, task_write],
    verbose=True
)

result = crew.kickoff(inputs={"question": "I want to hire 2 Software Engineer for a project. Find the cheapest and give me the details."})


print(f"Raw Output: {result.raw}")
print("-----------------------------------------\n\n")
print(f"Token Usage: {result.token_usage}")
print("-----------------------------------------\n\n")
print(f"Tasks Output of Task 1: {result.tasks_output[0]}")
print("-----------------------------------------\n\n")
print(f"Tasks Output of Task 2: {result.tasks_output[1]}")

In [18]:
import importlib.util

spec = importlib.util.find_spec("crewai_tools")
if spec is not None:
    print("The crewai module is installed.")
else:
    print("The crewai module is not installed.")

The crewai module is installed.


In [19]:
import sqlite3
print(sqlite3.version)

2.6.0


/var/folders/87/_sxm46r17nz31vwzz1ps6cpc0000gn/T/ipykernel_9562/4112040650.py:2: DeprecationWarning: version is deprecated and will be removed in Python 3.14
  print(sqlite3.version)
